In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import feedparser
import locale
import os
import pandas as pd
import requests
import tabula
from getRanges import getRanges

### Carica il feed .XML

In [2]:
url = 'https://www.regione.sicilia.it'
feed = feedparser.parse(url+'/feed')

### Per ogni entry che contiene la stringa nel titolo, selezionane il link

In [3]:
f = [field for field in feed['entries'] if "bollettino settimanale" in field['title']]
link = f[0]['links'][0]['href']
print(link)

http://www.regione.sicilia.it/la-regione-informa/covid-bollettino-settimanale-incidenza-stabile-picco-terze-dosi


### Chiama il link e parsa l'html con Soup, trova tutti gli hyperlinks della pagina

In [4]:
html = requests.get(link)
soup = BeautifulSoup(html.text, 'html.parser')
links = soup.find_all('a')

### Tra tutti i link, prendi quello con il .pdf

In [5]:
for link in links:
    if ('.pdf' in link.get('href')):
        pdf = url+link.get('href')

print(pdf)

https://www.regione.sicilia.it/sites/default/files/2021-11/Bollettino%20n%C3%82%C2%B0%205%20del%204%20novembre.pdf


### Estrae la data dal nome del PDF e la converte in YYYYMMDD

In [6]:
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')
date = pdf.rsplit('/',1)[-1].replace('%20',' ').lstrip('Bollettino').rstrip('.pdf')
date = date.lstrip('n%C3%82%C2%B0 5 del ')
date = datetime.strptime(date, '%d %B')
date = date.replace(year=2021)
#date = datetime.now()

print(date)

2021-11-04 00:00:00


### Scarica il PDF e aggiungi timestamp

In [7]:
r = requests.get(pdf, stream=True)
with open('./downloads/'+date.strftime('%Y%m%d')+'.pdf', 'wb') as f:
    f.write(r.content)
    print("PDF Scaricato.")

PDF Scaricato.


### Seleziona il piu recente PDF scaricato

In [8]:
files = []
path = './downloads/'

for file in os.listdir(path):
    if (file.endswith('.pdf')):
        files.append(file)

files.sort()
file = files[-1]
print(file)

20211104.pdf


### Quali pagine leggiamo?

In [9]:
ranges = getRanges(path+file)
vaccini = ranges['a2']
a2 = list(range(vaccini[0], vaccini[1]+1))
print(a2)

[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


### Leggi il PDF con tabula-py

In [10]:
print('Leggo PDF...attendi...')
pdf = tabula.read_pdf(path+file, pages=a2, pandas_options={'header': None}, multiple_tables=True, stream=True, silent=True)
print('Ho letto.')

Leggo PDF...attendi...
Ho letto.


### Unisci in un unico dataframe e bonifica i dati

In [11]:
vax = pd.concat(pdf).reset_index(drop=True)
vax = vax.dropna(thresh=3)
vax = vax[~vax[0].str.contains("Provincia", na=False)]
vax.drop(vax.columns[[0]], axis=1, inplace=True)

for index, row in vax.iterrows():
    if(pd.isnull(row[2])):
        row[2] = row[3]
        row[3] = row[4]

        
vax.reset_index(drop=True, inplace=True)

vax.drop(vax.columns[3], axis=1, inplace=True)

vax.columns = ['comune', '%vaccinati', '%immunizzati']

### Carica l'helper comuni siciliani

In [12]:
comuni = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/gabacode/vaxExtract/main/utilities/Elenco-comuni-siciliani.csv', converters={'pro_com_t': '{:0>6}'.format}))

### Inner join tra dataframes

In [13]:
out = pd.merge(vax, comuni, on='comune', how='inner')
out = out[['cod_prov','pro_com_t','provincia','comune','%vaccinati','%immunizzati']]

### 'Converti' percentuali a float e inserisci data

In [14]:
out['%vaccinati'] = out['%vaccinati'].str.replace(',','.').str.rstrip('%')
out['%immunizzati'] = out['%immunizzati'].str.replace(',','.').str.rstrip('%')
out.insert(0, 'data', date.strftime('%Y-%m-%d'))

### Seleziona target e calcola percentuali

In [15]:
target = pd.read_csv('https://raw.githubusercontent.com/opendatasicilia/comuni-italiani/main/dati/ISTAT_popolazione_2021.csv', converters={'pro_com_t': '{:0>6}'.format})
target = target[['pro_com_t','>=12']]
target.columns = ['pro_com_t', 'target']

final = pd.merge(out, target, on='pro_com_t', how='inner')

final['%vaccinati'] = (final['target'] * final['%vaccinati'].astype(float) / 100).astype(int)
final['%immunizzati'] = (final['target'] * final['%immunizzati'].astype(float) / 100).astype(int)

final.columns = ['data', 'cod_prov', 'pro_com_t', 'provincia', 'comune', 'vaccinati', 'immunizzati', 'target']

In [16]:
final

,data,cod_prov,pro_com_t,provincia,comune,vaccinati,immunizzati,target
0,2021-11-04,84,084001,Agrigento,Agrigento,44484,43118,50805
1,2021-11-04,84,084002,Agrigento,Alessandria della Rocca,2058,2015,2430
2,2021-11-04,84,084003,Agrigento,Aragona,7258,7070,8182
3,2021-11-04,84,084004,Agrigento,Bivona,2664,2606,3115
4,2021-11-04,84,084005,Agrigento,Burgio,2119,2087,2365
...,...,...,...,...,...,...,...,...
385,2021-11-04,81,081020,Trapani,San Vito Lo Capo,3274,3054,4381
386,2021-11-04,81,081019,Trapani,Santa Ninfa,3874,3743,4430
387,2021-11-04,81,081021,Trapani,Trapani,46973,44304,59041
388,2021-11-04,81,081022,Trapani,Valderice,8567,8125,10564


### Esporta in CSV

In [18]:
print("Converto in CSV.")
final.to_csv('./dati-csv/vaccini-'+date.strftime('%Y%m%d')+'.csv', index=None, header=True)
print("Fatto.")

Converto in CSV.
Fatto.
